#### **RAG Pipeline**

Retrieval-Augmented Generation or RAG is a method of improving the accuracy of LLM by supplementing base LLM with external knowledge sources such as files, database items, or documents. External data is stored as vector embeddings in vector databases that help in easy search and retrieval of data.

RAG is more resourceful and cost-effective in introducing new data to LLM, unlike fine-tuning or model retraining, which incorporates the additional knowledge into the model itself.

This notebook will help you to create a simple Q&A RAG flow using Qdrant vector database and a text file as our external data source. High level steps include:


1.   Create vector embeddings of the text file and load into our database.
2.   Retrieve a list of matches from the database for our query.
3.   Use this data to form a context.
4.   Create a prompt template with context and query.
5.   Run ChatModel with prompt




In [13]:
%%capture
!pip install langchain langchain-community langchain-openai
!pip install qdrant-client

In [14]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

In [15]:
key="sk-P............"

In [16]:
# load text doc
!wget "https://raw.githubusercontent.com/sacharya225/data-expts/master/RAG/state_of_the_union.txt"
loader = TextLoader("state_of_the_union.txt")
data = loader.load()

--2024-06-02 10:22:31--  https://raw.githubusercontent.com/sacharya225/data-expts/master/RAG/state_of_the_union.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39027 (38K) [text/plain]
Saving to: ‘state_of_the_union.txt.1’

state_of_the_union. 100%[===================>]  38.11K  --.-KB/s    in 0.01s   

2024-06-02 10:22:31 (3.50 MB/s) - ‘state_of_the_union.txt.1’ saved [39027/39027]



In [17]:
# split doc into chunks of size 1000
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(data)

# get OpenAI Embedding model
embeddings = OpenAIEmbeddings(api_key=key)


# upload vector embeddings into Qdrant
vectorstore = Qdrant.from_documents(docs, embeddings, location=":memory:", collection_name="sotu")

Retrieval and Generate step

In [18]:

query = 'What did the president say about Ketanji Brown Jackson'

# retrieve data that match query to form context .
docs_qdrant = vectorstore.similarity_search_with_score(query)

#above returns a list of tuples of matched Document and scores

# join all matched documents and generate a context/answer
context_text = "\n\n".join([doc.page_content for doc, score in docs_qdrant])
print(context_text)

Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. 

Tonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. 

One of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. 

And I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.

A former top litigator in private practice. A former federal public defender. And from a family of public school educators and police officers. A consensus builder. Since she’s been nominated, she’s received a b

In [19]:
# create/use a prompt template
template = """
You are an AI Assistant that follows instructions extremely well.
Assist with answers for questions based on the context: {context}
Use this context and answer the question : {question}."""

In [20]:
# get prompt template and load context and query
prompt_template = ChatPromptTemplate.from_template(template)
prompt = prompt_template.format(context=context_text, question=query)

# Invoke LLM chat model to generate answer based on the context and query
llm = ChatOpenAI(model="gpt-3.5-turbo-0125", api_key=key)
response = llm.invoke(prompt)
print(response.content)

The President praised Ketanji Brown Jackson as one of our nation's top legal minds who will continue Justice Breyer's legacy of excellence. He highlighted her background as a former top litigator, federal public defender, and consensus builder, who has received broad support since being nominated.
